<a href="https://colab.research.google.com/github/see-3pO/Learning_Tensorflow/blob/master/Model_Subclassing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # importing libraries
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.datasets import mnist

In [ ]:
# loading the data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# preprocessing the data by flattening and normalization
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

In [ ]:
# creating a CNN class
class CNNBlock(layers.Layer):
  def __init__(self, out_channels, kernel_size=3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding='same')
    self.bn = layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    x = self.conv(input_tensor)
    x = self.bn(x, training=training)
    x = tf.nn.relu(x)
    return x


In [ ]:
model = keras.Sequential(
    [
        CNNBlock(32),
        CNNBlock(64),
        CNNBlock(128),
        layers.Flatten(),
        layers.Dense(10),
    ]
)


In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'],
)

In [ ]:
model.fit(X_train, y_train, epochs=3, batch_size=64, verbose=2)

Epoch 1/3
938/938 - 703s - loss: 2.0906 - accuracy: 0.9366 - 703s/epoch - 749ms/step
Epoch 2/3
938/938 - 686s - loss: 0.2658 - accuracy: 0.9750 - 686s/epoch - 731ms/step
Epoch 3/3
938/938 - 688s - loss: 0.1329 - accuracy: 0.9816 - 688s/epoch - 733ms/step


In [ ]:
model.evaluate(X_test, y_test, batch_size=64, verbose=2)

157/157 - 29s - loss: 0.1705 - accuracy: 0.9760 - 29s/epoch - 187ms/step


[0.17046435177326202, 0.9760000109672546]

In [ ]:
# creating a ResNet Block class with 4 layers
class ResBlock(layers.Layer):
  def __init__(self, channels):
    super(ResBlock, self).__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(channels[2])
    self.pooling = layers.MaxPooling2D()
    self.identity_mapping = layers.Conv2D(channels[1], 1, padding='same')

  # foward pass
  def call(self, input_tensor, training=False):
    x = self.cnn1(input_tensor, training=training)
    x = self.cnn2(x, training=training)
    x = self.cnn3(
        x + self.identity_mapping(input_tensor), training=training,
    )
    return self.pooling(x)


In [ ]:
# building the final model
class ResNet_Like(keras.Model):
  def __init__(self, num_classes=10):
    super(ResNet_Like, self).__init__()
    self.block1 = ResBlock([32, 32, 64])
    self.block2 = ResBlock([128, 128, 256])
    self.block3 = ResBlock([128, 256, 512])
    self.pool = layers.GlobalAveragePooling2D()
    self.classifier = layers.Dense(num_classes)

  def call(self, input_tensor, training=False):
    x = self.block1(input_tensor, training=training)
    x = self.block2(x, training=training)
    x = self.block3(x, training=training)
    x = self.pool(x)
    return self.classifier(x)


In [ ]:
# instance of model
model1 = ResNet_Like(num_classes=10)

In [ ]:
model1.compile(
    optimizer = keras.optimizers.Adam(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'],
)

In [ ]:
model1.fit(X_train, y_train, epochs=1, batch_size=64, verbose=2)

938/938 - 1927s - loss: 0.0832 - accuracy: 0.9746 - 1927s/epoch - 2s/step


In [ ]:
print(model1.summary())